###### Imports

In [ ]:
import sympy as sp

sp.init_printing()
from dtumathtools import *
from sympy import plot


def inner(x1: sp.Matrix, x2: sp.Matrix):
    """
    Computes the inner product of two vectors of same length.
    """

    return x1.dot(x2, conjugate_convention="right")


sp.MutableDenseMatrix.inner = inner
sp.ImmutableDenseMatrix.inner = inner

# Taylor approksimering under forskellige betingelser

##### Følgende vil være scripts som kan taylor udvikle funktioner under forskellige betingelser, herunder hvis hele funktionen er givet, hvis dens partielt afledede er givet, og hvis den har flere variable. Den vil ligeledes plotte lortet sammenlignet med den originale funktion.

# Én variabel taylor approksimering

##### Hele funktionen er givet

In [ ]:
x = sp.symbols("x")

f = sp.sin(x) / x  # Indsæt din funktion her
x0 = 0  # Indsæt udviklingspunktet her
n = 3  # Indsæt graden af taylor polynomiet her (hvor mange gange, vi skal taylor udvikle)

P = sp.series(f, x, x0, n + 1).removeO()
display(P)

### Plot

##### Sammenlignet med taylor polynomiet

In [ ]:
taylor_approks_plot_sammenligning = sp.plot(
    f,
    P,
    (x, -2, 2),
    show=False,
    legend=True,
    title="Taylorudvikling af $f(x)$",
)

taylor_approks_plot_sammenligning.show()

##### Sammenlignet med de individuelle taylor udviklinger

In [ ]:
taylor_approks_plot_sammenligning_alle_udviklinger = sp.plot(
    f, xlim=(-3, 3), ylim=(-3, 3), show=False, legend=True
)

for i in range(n):
    newseries = sp.series(f, x, x0, i + 1).removeO()
    display(sp.Eq(sp.Function(f"P_{i}")(x), newseries))
    newplot = sp.plot(newseries, label=f"n = {i}", show=False)
    taylor_approks_plot_sammenligning_alle_udviklinger.extend(newplot)

taylor_approks_plot_sammenligning_alle_udviklinger.show()

# Én variabel piecewise funktion taylor approksimering

F.eks. er følgende stykvis funktion givet:

\begin{equation*}
g(0)=1  \quad \text{og} \quad  g^{(n)} (0)=  
\begin{cases}  
  (-1)^{\frac{n}{2}}2^n & n \text{ er lige}, \\
  0 &  n  \text{ er ulige}.
\end{cases}  
\end{equation*}

Her ved vi med det samme hvad $g(0)$ er, men $g(x)$ er lidt fucked. Den skriver vi ind som funktion, og bagefter taylorudvikler vi bare helt slavisk. Her taylorudvikler vi 7 gange, men du kan tilpasse funktionen til opgaven. Mønsteret er tydeligt at se.

In [ ]:
x0 = 0  # Udviklingspunkt
gx0 = 1


def g(n):
    if n % 2 == 0:
        return (-1) ** (n / 2) * 2**n
    else:
        return 0


P7_g = (
    g(0)
    + g(1) * (x - x0)
    + g(2) * (x - x0) ** 2 / sp.factorial(2)
    + g(3) * (x - x0) ** 3 / sp.factorial(3)
    + g(4) * (x - x0) ** 4 / sp.factorial(4)
    + g(5) * (x - x0) ** 5 / sp.factorial(5)
    + g(6) * (x - x0) ** 6 / sp.factorial(6)
    + g(7) * (x - x0) ** 7 / sp.factorial(7)
)


display(P7_g)

# To variabler taylor approksimering

##### Hele funktionen er givet

> ###### FEATURING CHATGPT

In [ ]:
def taylor_approximation_2_variables(func, variabler, udviklingspunkt, order):

    # Destructure variables and expansion point
    x, y = variabler
    x0, y0 = udviklingspunkt

    # Compute the Taylor series
    taylor_series = sp.series(func, x, x0, order + 1).removeO()
    taylor_series = sp.series(taylor_series, y, y0, order + 1).removeO()

    return taylor_series


# Example usage
x, y = sp.symbols("x y")
func = sp.sin(x) + sp.cos(y)
udviklingspunkt = (0, 0)
orden = 2

approx = taylor_approximation_2_variables(func, (x, y), udviklingspunkt, orden)
display(approx)

Det kan gøres mindre cursed, og mere slavisk, hvis man gør det på den "rigtige" måde. Så er skabelonen her.

Først skal vi beregne funktionens partielt afledte. Dette skal gøres for både 1. og 2. ordens partielt afledte for $x$ og $y$ samt den partielt afledte for både $x$ og $y$ sammen.

In [ ]:
x, y = sp.symbols("x y")  # HUSK: tilpass dette om nødvendigt

f = sp.sin(x) + sp.cos(y)  # Definer din funktion af to variable

dfx = sp.diff(f, x)  # 1. partielt afledte for x
dfy = sp.diff(f, y)  # 1. partielt afledte for y

ddfxx = sp.diff(dfx, x)  # 2. partielt afledte for x
ddfyy = sp.diff(dfy, y)  # 2. partielt afledte for y

ddfxy = sp.diff(dfx, y)  # 2. partielt afledte for x og y

Herefter skal vi evaluere alle disse afledte i udviklingspunktet. Det gør vi sådan her:

###### Det er lidt cursed syntaks men jeg har også selv skrevet det så hvad havde I regnet med hahahah

In [ ]:
udviklingspunkt = (0, 0)  # Angiv udviklingspunktet her


a, b = udviklingspunkt

evaluated_f = f.subs({x: a, y: b})
evaluated_dfx = dfx.subs({x: a, y: b})
evaluated_dfy = dfy.subs({x: a, y: b})
evaluated_ddfxx = ddfxx.subs({x: a, y: b})
evaluated_ddfyy = ddfyy.subs({x: a, y: b})
evaluated_ddfxy = ddfxy.subs({x: a, y: b})

for evalueringer in [
    evaluated_f,
    evaluated_dfx,
    evaluated_dfy,
    evaluated_ddfxx,
    evaluated_ddfyy,
    evaluated_ddfxy,
]:
    display(evalueringer)

Nu har vi kun tilbage at konstruere selve taylor approksimationen:

In [ ]:
# Dette er blot den cursed taylor formel for to variable
taylor_approx = (
    evaluated_f
    + evaluated_dfx * (x - a)
    + evaluated_dfy * (y - b)
    + (
        evaluated_ddfxx * (x - a) ** 2
        + 2 * evaluated_ddfxy * (x - a) * (y - b)
        + evaluated_ddfyy * (y - b) ** 2
    )
)

taylor_approx

## To variablere 4. orden taylor approksimation

> ###### FEATURING CHATGPT

> ###### dunno tænkte nu har jeg den, might as well inkludere den

In [ ]:
x, y = sp.symbols("x y")
f = sp.sin(x) + sp.cos(y)

# Define the point of expansion
a, b = 0, 0

# Calculate derivatives up to the fourth order
derivatives = {}
for i in range(5):  # up to and including 4th derivatives
    for j in range(i + 1):
        d = sp.diff(f, x, j, y, i - j)
        derivatives[(j, i - j)] = d

# Evaluate each derivative at (a, b)
evaluated_derivatives = {k: v.subs({x: a, y: b}) for k, v in derivatives.items()}

# Construct the Taylor series
taylor_series = f.subs({x: a, y: b})
for k in range(1, 5):  # from 1st to 4th order terms
    taylor_series += sum(
        sp.Rational(1, sp.factorial(j) * sp.factorial(k - j))
        * evaluated_derivatives[(j, k - j)]
        * (x - a) ** j
        * (y - b) ** (k - j)
        for j in range(k + 1)
    )

# Simplify the series for better readability
taylor_series = sp.simplify(taylor_series)
display(taylor_series)

# To variable, har kun de to 1. ordens partielt afledede

Hvis der kommer en opgave, hvor du kun får givet de 1. ordens partielt afledte af en funktion, og altså IKKE hele funktionen, så kan du bruge dette script til at taylor udvikle lortet.

In [ ]:
# Define the variables
x, y = sp.symbols("x y")

# Integrate dfx with respect to x, assume integration constant is a function of y: g(y)
f_from_dfx = sp.integrate(6 * x - 6 * y, x) + sp.Function("g")(y)

# Integrate dfy with respect to y, assume integration constant is a function of x: h(x)
f_from_dfy = sp.integrate(6 * y**2 - 6 * x, y) + sp.Function("h")(x)

# Display both to check if they can provide insight into f(x, y)
f_from_dfx, f_from_dfy